**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [2]:
which bedtools
bedtools -version

/nfs/software/helmod/apps/Core/bedtools2/2.25.0-fasrc01/bin/bedtools
bedtools v2.25.0


In [3]:
ls -d ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log
all


**TEST**

In [4]:
FD_REG=${FD_RES}/region/KS91_K562_astarrseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
ls -lh ${FD_REG}/${FN_REG}
cat ${FD_REG}/${FN_REG} | wc -l

-rw-rw-r-- 1 kk319 reddylab 5.7M Oct 17 05:28 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
247520


In [5]:
PREFIX=KS91_K562_ASTARRseq
FD_CNT=${FD_RES}/${PREFIX}/fragment_count
ls ${FD_CNT}/*.unstranded.bed | xargs -n 1 basename
ls ${FD_CNT}/*.unstranded.bed | wc -l

KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.MYC.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.FADS.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.MYC.unstran

**RUN**

In [9]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.%a.txt \
    --array 0-29 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output directory
PREFIX=KS91_K562_ASTARRseq
FD_CNT=${FD_RES}/${PREFIX}/fragment_count
FD_COV=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input

FP_INPS=($(ls ${FD_CNT}/*.unstranded.bed))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FD_OUT=${FD_COV}
FN_OUT=${FN_INP}.gz

FD_REG=${FD_RES}/region/KS91_K562_astarrseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
cat  ${FP_INP} | head -n 3 
echo

echo "Input  file: " ${FD_REG}/${FN_REG}
echo
echo "Show the first few lines of the input file"
cat  ${FD_REG}/${FN_REG} | head -n 3 
echo

### execute
bedtools map \
    -a ${FD_REG}/${FN_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FD_OUT}/${FN_OUT}

### Show I/O file
echo
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the output file"
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29168542


**CHECK**

In [7]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.0.txt

Hostname:           x2-04-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-21-22+14:23:06

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_count/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed

Show the first few lines of the input file
chr11	61787228	61787382	chr11_61787228_61787382	1	.
chr11	61787230	61787628	chr11_61787230_61787628	1	.
chr11	61787234	61787471	chr11_61787234_61787471	1	.

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477


Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed.gz

Show the first few lines of the output file
chr1	10015	10442	

In [8]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.29.txt

Hostname:           x1-03-1.genome.duke.edu
Slurm Array Index:  29
Time Stamp:         10-21-22+14:23:37

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_count/KS91_K562_hg38_ASTARRseq_Output_rep4.MYC.unstranded.bed

Show the first few lines of the input file
chr8	126736025	126736220	chr8_126736025_126736220	5	.
chr8	126736089	126736259	chr8_126736089_126736259	1	.
chr8	126736091	126736273	chr8_126736091_126736273	9	.

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477


Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Output_rep4.MYC.unstranded.bed.gz

Show the first few lines of the output file
chr1	10015

In [8]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.0.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-17-22+10:11:28

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_count/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed

Show the first few lines of the input file
chr11	61787228	61787382	chr11_61787228_61787382	1	.
chr11	61787230	61787628	chr11_61787230_61787628	1	.
chr11	61787234	61787471	chr11_61787234_61787471	1	.

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477


Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed.gz

Show the first few lines of the output file
chr1	10015	10442	

In [9]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.29.txt

Hostname:           x1-03-3.genome.duke.edu
Slurm Array Index:  29
Time Stamp:         10-17-22+10:11:28

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/fragment_count/KS91_K562_hg38_ASTARRseq_Output_rep4.MYC.unstranded.bed

Show the first few lines of the input file
chr8	126736025	126736220	chr8_126736025_126736220	5	.
chr8	126736089	126736259	chr8_126736089_126736259	1	.
chr8	126736091	126736273	chr8_126736091_126736273	9	.

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_astarrseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477


Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Output_rep4.MYC.unstranded.bed.gz

Show the first few lines of the output file
chr1	10015

In [10]:
PREFIX=KS91_K562_ASTARRseq
FDIRY=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input
FNAME=KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed.gz
FPATH=${FDIRY}/${FNAME} 
echo ${FPATH}
zcat ${FPATH} | grep chrX | head -n 1400 | tail -n 4

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed.gz
grep: write error
chrX	47809119	47809445	92
chrX	47814810	47815443	251
chrX	47816459	47818070	543
chrX	47836113	47837157	2426

gzip: stdout: Broken pipe


In [11]:
PREFIX=KS91_K562_ASTARRseq
FDIRY=${FD_RES}/${PREFIX}/coverage_astarrseq_peak_macs_input
FNAME=KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed.gz
FPATH=${FDIRY}/${FNAME} 

zcat ${FPATH} | grep chr8 | head -n 8300 | tail -n 4

chr8	127793817	127795585	1506
chr8	127800218	127801557	1779
chr8	127806610	127807136	134
chr8	127807462	127808467	222
grep: write error

gzip: stdout: Broken pipe


In [12]:
ls ${FDIRY}

KS91_K562_hg38_ASTARRseq_Input_rep1.FADS.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep1.MYC.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep2.FADS.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep2.GATA1.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep2.MYC.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep3.FADS.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep3.GATA1.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep3.MYC.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep4.FADS.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep4.MYC.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep5.FADS.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep5.GATA1.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep5.MYC.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep6.FADS.unstranded.bed.gz
KS91_K562_hg38_ASTARRseq_Input_rep6.GATA1.unstranded.bed